In [409]:
import pandas as pd

# import semi_production_and_use.xlsx sheet
semi_production_and_use = pd.read_excel('semi_production_and_use.xlsx', sheet_name='production', header=3)
#drop the first 9 rows
semi_production_and_use = semi_production_and_use.drop(semi_production_and_use.index[0:9])
#drop the last 4 rows
semi_production_and_use = semi_production_and_use.drop(semi_production_and_use.index[-4:])
# rename Class 8 Tractor Regional Haul to regional
semi_production_and_use = semi_production_and_use.rename(columns={'Class 8 Tractor Regional Haul': 'regional'})
# rename Class 8 Tractor Long Haul to long
semi_production_and_use = semi_production_and_use.rename(columns={'Class 8 Tractor Long Haul': 'long'})
# select year, regional, and long columns
semi_production_and_use = semi_production_and_use[['regional', 'long']]
# create short column as 0.1 * regional
semi_production_and_use['short'] = 0.1 * semi_production_and_use['regional']
semi_production_and_use

# convert df to dict
semi_production_and_use_dict = semi_production_and_use.to_dict('records')
semi_production_and_use_dict

data = [{'diesel': x, 'electric': {'short': 0, 'regional': 0, 'long': 0}} for x in semi_production_and_use_dict]

diesel_data = [x['diesel'] for x in data]
#convert diesel_data to df
diesel_data_df = pd.DataFrame(diesel_data)
diesel_data_df

,regional,long,short
0,35505.0,57930.0,3550.5
1,23009.0,46715.0,2300.9
2,37798.0,44372.0,3779.8
3,74237.0,71325.0,7423.7
4,74361.0,80558.0,7436.1
5,70667.0,67896.0,7066.7
6,103295.0,74799.0,10329.5
7,107995.0,95769.0,10799.5
8,68355.0,55927.0,6835.5
9,68086.0,70866.0,6808.6


In [410]:
hauls = ['short', 'regional', 'long']

diesel_price_ratios = {
    'short': 0.71315653278,
    'regional': 0.76366322008,
    'long': 0.810823121
}

total_trucks = 1734721 / 0.95

truck_counts = data.copy()

stay = 0.91
on_the_fence = 0.08
revert = 0.01

for i in range(20):
    count_next = {'electric': {}, 'diesel': {}}
    for haul in hauls:
        decommissioned_diesel = truck_counts[-12]['diesel'][haul]
        decommissioned_electric = truck_counts[-12]['electric'][haul]

        produced_electric = decommissioned_diesel * diesel_price_ratios[haul] + decommissioned_electric * stay + diesel_price_ratios[haul]* decommissioned_electric*on_the_fence
        produced_diesel = decommissioned_electric * revert + (1 - diesel_price_ratios[haul])*decommissioned_electric*on_the_fence + decommissioned_diesel * (1 - diesel_price_ratios[haul])

        new_electric = produced_electric - produced_diesel
        new_diesel = -new_electric

        count_next['electric'][haul] = new_electric
        count_next['diesel'][haul] = new_diesel

    truck_counts.append(count_next)
    truck_counts

In [411]:
electric_output = [x['electric'] for x in truck_counts]

#convert output to df
electric_output_df = pd.DataFrame(electric_output)

# create new column for each haul in hauls that sums the previous 12 rows of that haul
for haul in hauls:
    electric_output_df[haul + '_sum'] = electric_output_df[haul].rolling(12).sum()

# rename all columns to have the prefix electric_
electric_output_df = electric_output_df.add_prefix('electric_')

# drop first 11 rows
electric_output_df = electric_output_df.drop(electric_output_df.index[0:11])

electric_output_df

,electric_short,electric_regional,electric_long,electric_short_sum,electric_regional_sum,electric_long_sum
11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12,1513.624539,18722.725258,36011.966799,1513.624539,18722.725258,36011.966799
13,980.903733,12133.254062,29040.204195,2494.528272,30855.979320,65052.170994
14,1611.378125,19931.884785,27583.687050,4105.906397,50787.864105,92635.858044
15,3164.820305,39147.132938,44338.918211,7270.726702,89934.997043,136974.776255
16,3170.106587,39212.521417,50078.577963,10440.833289,129147.518460,187053.354218
17,3012.626540,37264.577547,42207.293247,13453.459829,166412.096006,229260.647465
18,4403.600811,54470.184636,46498.517255,17857.060640,220882.280643,275759.164720
19,4603.967952,56948.618905,59534.438950,22461.028591,277830.899548,335293.603670
20,2914.062960,36045.398817,34766.809376,25375.091551,313876.298365,370060.413046


In [412]:
diesel_output = [x['diesel'] for x in truck_counts]

#convert output to df
diesel_output_df = pd.DataFrame(diesel_output)

# create new column for each haul in hauls that sums the previous 12 rows of that haul, then drop that haul column
for haul in hauls:
    diesel_output_df[haul + '_sum'] = diesel_output_df[haul].rolling(12).sum()

# rename all columns to have the prefix diesel_
diesel_output_df = diesel_output_df.add_prefix('diesel_')

# drop the first 11 columns
diesel_output_df = diesel_output_df.drop(diesel_output_df.index[0:11])

diesel_output_df

,diesel_regional,diesel_long,diesel_short,diesel_short_sum,diesel_regional_sum,diesel_long_sum
11,109442.000000,101024.000000,10944.200000,87598.900000,875989.000000,858732.000000
12,-18722.725258,-36011.966799,-1513.624539,82534.775461,821761.274742,764790.033201
13,-12133.254062,-29040.204195,-980.903733,79252.971728,786619.020680,689034.829006
14,-19931.884785,-27583.687050,-1611.378125,73861.793603,728889.135895,617079.141956
15,-39147.132938,-44338.918211,-3164.820305,63273.273298,615505.002957,501415.223745
16,-39212.521417,-50078.577963,-3170.106587,52667.066711,501931.481540,370778.645782
17,-37264.577547,-42207.293247,-3012.626540,42587.740171,393999.903994,260675.352535
18,-54470.184636,-46498.517255,-4403.600811,27854.639360,236234.719357,139377.835280
19,-56948.618905,-59534.438950,-4603.967952,12451.171409,71291.100452,-15925.603670
20,-36045.398817,-34766.809376,-2914.062960,2701.608449,-33109.298365,-106619.413046


In [413]:
# merge diesel_out_df and electric_out_df
output_df = pd.concat([diesel_output_df, electric_output_df], axis=1)

# create new column with sums of diesel_short_sum, diesel_regional_sum, and diesel_long_sum, then drop those 3 columns
output_df['diesel_sum'] = output_df['diesel_short_sum'] + output_df['diesel_regional_sum'] + output_df['diesel_long_sum']


# create new column with sums of electric_short_sum, electric_regional_sum, and electric_long_sum, then drop those 3 columns
output_df['electric_sum'] = output_df['electric_short_sum'] + output_df['electric_regional_sum'] + output_df['electric_long_sum']

# create new column called diesel_prop with diesel_sum/(diesel_sum + electric_sum)
output_df['diesel_prop'] = output_df['diesel_sum']/(output_df['diesel_sum'] + output_df['electric_sum'])
output_df['electric_prop'] = output_df['electric_sum']/(output_df['diesel_sum'] + output_df['electric_sum'])

output_df

,diesel_regional,diesel_long,diesel_short,diesel_short_sum,diesel_regional_sum,diesel_long_sum,electric_short,electric_regional,electric_long,electric_short_sum,electric_regional_sum,electric_long_sum,diesel_sum,electric_sum,diesel_prop,electric_prop
11,109442.000000,101024.000000,10944.200000,87598.900000,875989.000000,858732.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.822320e+06,0.000000e+00,1.000000e+00,0.000000e+00
12,-18722.725258,-36011.966799,-1513.624539,82534.775461,821761.274742,764790.033201,1513.624539,18722.725258,36011.966799,1513.624539,18722.725258,36011.966799,1.669086e+06,5.624832e+04,9.673986e-01,3.260140e-02
13,-12133.254062,-29040.204195,-980.903733,79252.971728,786619.020680,689034.829006,980.903733,12133.254062,29040.204195,2494.528272,30855.979320,65052.170994,1.554907e+06,9.840268e+04,9.404814e-01,5.951861e-02
14,-19931.884785,-27583.687050,-1611.378125,73861.793603,728889.135895,617079.141956,1611.378125,19931.884785,27583.687050,4105.906397,50787.864105,92635.858044,1.419830e+06,1.475296e+05,9.058738e-01,9.412621e-02
15,-39147.132938,-44338.918211,-3164.820305,63273.273298,615505.002957,501415.223745,3164.820305,39147.132938,44338.918211,7270.726702,89934.997043,136974.776255,1.180194e+06,2.341805e+05,8.344282e-01,1.655718e-01
16,-39212.521417,-50078.577963,-3170.106587,52667.066711,501931.481540,370778.645782,3170.106587,39212.521417,50078.577963,10440.833289,129147.518460,187053.354218,9.253772e+05,3.266417e+05,7.391080e-01,2.608920e-01
17,-37264.577547,-42207.293247,-3012.626540,42587.740171,393999.903994,260675.352535,3012.626540,37264.577547,42207.293247,13453.459829,166412.096006,229260.647465,6.972630e+05,4.091262e+05,6.302149e-01,3.697851e-01
18,-54470.184636,-46498.517255,-4403.600811,27854.639360,236234.719357,139377.835280,4403.600811,54470.184636,46498.517255,17857.060640,220882.280643,275759.164720,4.034672e+05,5.144985e+05,4.395232e-01,5.604768e-01
19,-56948.618905,-59534.438950,-4603.967952,12451.171409,71291.100452,-15925.603670,4603.967952,56948.618905,59534.438950,22461.028591,277830.899548,335293.603670,6.781667e+04,6.355855e+05,9.641236e-02,9.035876e-01
20,-36045.398817,-34766.809376,-2914.062960,2701.608449,-33109.298365,-106619.413046,2914.062960,36045.398817,34766.809376,25375.091551,313876.298365,370060.413046,-1.370271e+05,7.093118e+05,-2.394387e-01,1.239439e+00
